###### Install Selenium

In [0]:
%pip install selenium

In [0]:
import pickle as pkl
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options

###### Download the latest ChromeDriver to the DBFS root storage /tmp/. The curl command will get the latest Chrome version and store in the version variable. %sh command allows us to execute shell commands on the driver. Dont put space.

In [ ]:
%sh
version=`curl -sS https://chromedriver.storage.googleapis.com/LATEST_RELEASE`
wget -N https://chromedriver.storage.googleapis.com/${version}/chromedriver_linux64.zip -O /tmp/chromedriver_linux64.zip

###### Unzip the file to a new folder in the DBFS root /tmp/.

In [0]:
%sh
unzip /tmp/chromedriver_linux64.zip -d /tmp/chromedriver/

Archive: /tmp/chromedriver_linux64.zip
 inflating: /tmp/chromedriver/chromedriver

###### Get the latest Chrome download and install it.

In [ ]:
%sh
sudo curl -sS -o - https://dl-ssl.google.com/linux/linux_signing_key.pub | apt-key add
sudo echo "deb https://dl.google.com/linux/chrome/deb/ stable main" >> /etc/apt/sources.list.d/google-chrome.list
sudo apt-get -y update
sudo apt-get -y install google-chrome-stable

##### Mounting to the Blob container

In [0]:
containerName = "kaufland-ads"
storageAccountName = "advertising"
sas = "sp=racwdli&st=2022-08-30T11:1.......ljme%2B3t8GwvuMX2KDq6UtVQ%3D"
config = "fs.azure.sas." + containerName+ "." + storageAccountName + ".blob.core.windows.net"
mount_point = "/mnt/kaufland-ads"
utils_folder = mount_point + "/utils/selenium/"
raw_folder = mount_point + "/raw/"

if not any(mount_point in mount_info for mount_info in dbutils.fs.mounts()):
    dbutils.fs.mount( source = "wasbs://{}@{}.blob.core.windows.net".format(containerName,storageAccountName),
                      mount_point = mount_point,
                      extra_configs = {config : sas}
                    )
    print(f"{mount_point} was mounted.")

else:
    print(mount_point + " was already mounted.")
    print(f"Utils folder: {utils_folder}")
    print(f"Raw folder: {raw_folder}")

/mnt/kaufland-ads was already mounted.
Utils folder: /mnt/kaufland-ads/utils/selenium/
Raw folder: /mnt/kaufland-ads/raw/

##### Create method for Chrome browser

In [0]:
# If the url needs cookies, then put cookies in a pickle in related path as pickle file
def init_chrome_browser(download_path, chrome_driver_path, cookies_path, url): 
    options = Options()
    now = datetime.now()
    prefs = {'download.default_directory' : download_path}
    options.add_experimental_option('prefs', prefs)
    options.add_argument('--no-sandbox')
    options.add_argument('--headless')
    options.add_argument('--disable-dev-shm-usage')
    options.add_argument('--start-maximized')
    options.add_argument('window-size=2560,1440')
    print(f"{now}    Launching Chrome...")
    browser = webdriver.Chrome(service=Service(chrome_driver_path), options=options)
    print(f"{now}    Chrome launched.")
    browser.get(url)
    #print(f"{now}    Loading cookies...")
    #cookies = pkl.load(open(cookies_path, "rb"))
    #for cookie in cookies:
    #    browser.add_cookie(cookie)
    browser.get(url)
    print(f"{now}    Cookies loaded.")
    print(f"{now}    Browser ready to use.")
    return browser

##### Open Chrome browser

In [0]:
browser = init_chrome_browser(
                              download_path = "/tmp/downloads",
                              chrome_driver_path = "/tmp/chromedriver/chromedriver",
                              cookies_path = "/dbfs"+ utils_folder + "cookies.pkl",
                              url = "https://sellerportal.kaufland.de/login"
                             )

2022-08-30 13:39:30.192227 Launching Chrome...
2022-08-30 13:39:30.192227 Chrome launched.
2022-08-30 13:39:30.192227 Cookies loaded.
2022-08-30 13:39:30.192227 Browser ready to use.